In [23]:
import geopandas as gpd
import json
import os
import requests
import urllib

from datetime import datetime
from pathlib import Path
from typing import Tuple

In [24]:
def shapefile_to_bbox(shapefile_path: Path, target_epsg: int = 4326) -> Tuple[float, float, float, float]:
    """
    Read a shapefile, reproject to the target CRS, and return its bounding box.

    Returns:
        (minx, miny, maxx, maxy) in the target CRS.
    """
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.to_crs(epsg=target_epsg)
    minx, miny, maxx, maxy = gdf.total_bounds
    return minx, miny, maxx, maxy

# path where annotations are located
ANNOTATIONS_PATH = Path("../landslides-detection-main/inventories/")
LOMBOK_PATH = ANNOTATIONS_PATH / "Lombok2018"
PHILIPPINES_PATH = ANNOTATIONS_PATH / "Philippines2019"
EMILIA_PATH = ANNOTATIONS_PATH / "EmiliaRomagna2023"
MICHOACAN_PATH = ANNOTATIONS_PATH / "Michoacan2022"

# inventories list
INVENTORIES = [
    {
        "name": "Lombok2018",
        "area": LOMBOK_PATH / "area.shp",
        "dates": [datetime(2018, 9, 9), datetime(2018, 9, 13), datetime(2018, 9, 27)],
    },
    {
        "name": "Philippines2019",
        "area": PHILIPPINES_PATH / "area_study.shp",
        "dates": [datetime(2020, 5, 2)],
    },
    {
        "name": "Michoacan2022",
        "area": EMILIA_PATH / "Area_rev.shp",
        "dates": [datetime(2022, 9, 25), datetime(2022, 10, 31)],
    },
    {
        "name": "EmiliaRomagna2023",
        "area": MICHOACAN_PATH / "investigated_area_LS_Michoacan2022.shp",
        "dates": [datetime(2023, 5, 22)],
    },
]


In [25]:
with open("./key.txt", "r") as f:
    API_key = f.read()

## Testing

In [26]:
API_URL = "https://api.planet.com/basemaps/v1/mosaics"
session = requests.Session()
session.auth = (API_key, "")

# get list of available mosaics
response = session.get(API_URL)
data = response.json()

In [27]:
BASE_URL = "https://api.planet.com/basemaps/v1/mosaics?api_key={}"
res = requests.get(BASE_URL.format(API_key))

for i, _ in enumerate(res.json()):
    print(res.json()["mosaics"][i]["name"])

global_monthly_2016_01_mosaic
global_monthly_2016_02_mosaic


In [28]:
y = "2016"
m = "03"

params = {
    "name__is" :f"global_monthly_{y}_{m}_mosaic"
}

res = session.get(API_URL, params = params)
print(res.status_code)

mosaic = res.json()
#print(json.dumps(mosaic, indent=2))

200


In [29]:
#get id
mosaic_id = mosaic['mosaics'][0]['id']
#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']
#converting bbox to string for search params
string_bbox = ','.join(map(str, mosaic_bbox))

print('Mosaic id: '+ mosaic_id)
print('Mosaic bbox: '+ string_bbox)

Mosaic id: cfd6a60f-ec1e-491d-a885-e06d43990bd5
Mosaic bbox: -180,-55,180,76


## Solution - bbox coordinates are not set correctly

With valid coordinates, you will receive a list of items.
The download process then iterates through each item, retrieves its link, and downloads the corresponding content.

To select the month and year, change the "name__is" parameter.

In [30]:
y = "2018"
m = "09"

# Get bounding box as tuple and convert to string
area_bbox = shapefile_to_bbox(INVENTORIES[0]["area"])  # Returns (minx, miny, maxx, maxy)
area_bbox_str = ",".join(map(str, area_bbox))

print("BBox:", area_bbox_str)

# Prepare API request parameters
search_parameters = {
    "name__is": f"global_monthly_{y}_{m}_mosaic",
    "bbox": area_bbox_str,
    "level": str(18),
}

quads_url = f"{API_URL}/{mosaic_id}/quads"

# Send request
res = session.get(quads_url, params=search_parameters, stream=True)

# Handle response
print("Status code:", res.status_code)
try:
    data = res.json()
    #print("Response:", json.dumps(data, indent=2))

    if "items" in data:
        items = data["items"]
        #print("First quad item:\n", json.dumps(items[0], indent=2))
    else:
        print("No quads found for the given bbox.")
except Exception as e:
    print("Error parsing JSON:", e)

BBox: 116.03246503532793,-8.56332060429984,116.70731520051434,-8.257044874009
Status code: 200


In [31]:
# download
# a folder named "quads" has to be created beforehand
for i in items:
    link = i['_links']['download']
    name = i['id']
    name = name + '.tiff'
    DIR = 'quads/test'
    filename = os.path.join(DIR, name)

    # remove the comment to actually download images
    if not os.path.isfile(filename):
        pass
        #urllib.request.urlretrieve(link, filename)

# Automated

In [32]:
def get_previous_month(month_str, year_str):
    month = int(month_str)
    year = int(year_str)

    if month == 1:
        prev_month = 12
        prev_year = year - 1
    else:
        prev_month = month - 1
        prev_year = year

    return f"{prev_month:02}", str(prev_year)


def get_next_month(month_str, year_str):
    month = int(month_str)
    year = int(year_str)

    if month == 12:
        next_month = 1
        next_year = year + 1
    else:
        next_month = month + 1
        next_year = year

    return f"{next_month:02}", str(next_year)

In [33]:
for el in INVENTORIES:
    dates = el["dates"]

    # gets the date before the first date and after the last date
    if len(dates) == 1:
        year = dates[0].year
        month = dates[0].month

        month_before, year_before = get_previous_month(month, year)
        month_after, year_after = get_next_month(month, year)
    else:
        first_year = dates[0].year
        last_year = dates[-1].year
        first_month = dates[0].month
        last_month = dates[-1].month
        
        month_before, year_before = get_previous_month(first_month, first_year)
        month_after, year_after = get_next_month(last_month, last_year)
    
    print(month_before, year_before)
    print(month_after, year_after)
    
    # get bounding box as tuple and convert to string
    area_bbox = shapefile_to_bbox(el["area"])  # Returns (minx, miny, maxx, maxy)
    area_bbox_str = ",".join(map(str, area_bbox))
    
    #print("BBox:", area_bbox_str)
    
    # prepare API request parameters
    search_parameters_before = {
        "name__is": f"global_monthly_{year_before}_{month_before}_mosaic",
        "bbox": area_bbox_str,
        "minimal": True
    }
    
    search_parameters_after = {
        "name__is": f"global_monthly_{year_after}_{month_after}_mosaic",
        "bbox": area_bbox_str,
        "minimal": True
    }
    
    quads_url = f"{API_URL}/{mosaic_id}/quads"
    
    # send request (before)
    res = session.get(quads_url, params=search_parameters_before, stream=True)
    
    # handle response
    print("Status code:", res.status_code)
    try:
        data = res.json()
        #print("Response:", json.dumps(data, indent=2))
    
        if "items" in data:
            items = data["items"]
            #print("First quad item:\n", json.dumps(items[0], indent=2))
        else:
            print("No quads found for the given bbox.")
    except Exception as e:
        print("Error parsing JSON:", e)

    # download images (before)
    for i in items:
        link = i['_links']['download']
        name = i['id']
        name = name + '_before' + '.tiff'
        DIR = 'quads/' + el["name"]

        os.makedirs(DIR, exist_ok=True)
        
        filename = os.path.join(DIR, name)
        
        if not os.path.isfile(filename):
            urllib.request.urlretrieve(link, filename)

    # send request (after)
    res = session.get(quads_url, params=search_parameters_after, stream=True)
    
    # handle response
    print("Status code:", res.status_code)
    try:
        data = res.json()
        #print("Response:", json.dumps(data, indent=2))
    
        if "items" in data:
            items = data["items"]
            #print("First quad item:\n", json.dumps(items[0], indent=2))
        else:
            print("No quads found for the given bbox.")
    except Exception as e:
        print("Error parsing JSON:", e)

    # download images (after)
    for i in items:
        link = i['_links']['download']
        name = i['id']
        name = name + '_after' + '.tiff'
        DIR = 'quads/' + el["name"]

        os.makedirs(DIR, exist_ok=True)
        
        filename = os.path.join(DIR, name)
        
        if not os.path.isfile(filename):
            urllib.request.urlretrieve(link, filename)

08 2018
10 2018
Status code: 200
Status code: 200
04 2020
06 2020
Status code: 200
Status code: 200
08 2022
11 2022
Status code: 200
Status code: 200
04 2023
06 2023
Status code: 200
Status code: 200
